In [4]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [5]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineAnalysis").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [7]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
|R3KFIK8P0I91PL|          5|            0|          0|   N|                Y|
| RENOAY76PPK1O|          5|            0|          0|   N|                Y|
| RINV884I0NL5V|          1|            0|          0|   N|                Y|
| R5KJH6CXZH2PX|          5|            0|          0|   N|                Y|
| RO69JF6QWD0W1|          5|            0|          0|   N|     

In [8]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
filtered_df = df.filter("total_votes>20")
filtered_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47749608| RXAHWIC1584UQ|B00NKSMPZW|     824618679|ClearArmor 141001...|           Tools|          5|           20|         23|   N|                Y|          Five Stars|geart product :) ...| 2015-08-31|
|         US|   53082915| RSBELAIC899DO|B00LPFIPPQ|     848632692|WEN 6510 Oscillating|           Tools|          5|    

In [9]:
#Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
helpful_df = filtered_df.filter("helpful_votes / total_votes > 0.50")
helpful_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47749608| RXAHWIC1584UQ|B00NKSMPZW|     824618679|ClearArmor 141001...|           Tools|          5|           20|         23|   N|                Y|          Five Stars|geart product :) ...| 2015-08-31|
|         US|   53082915| RSBELAIC899DO|B00LPFIPPQ|     848632692|WEN 6510 Oscillating|           Tools|          5|    

In [10]:
#Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_df = helpful_df.filter(helpful_df.vine == "Y")
paid_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21084288|R3QIQRG107AP19|B00OJ72LHK|     369815253|BLACK+DECKER BDEJ...|           Tools|          5|           38|         41|   Y|                N|Essential home ow...|We are not pros. ...| 2015-08-27|
|         US|   52512530|R1ZNBHGCYE8VN1|B00OJ72LHK|     369815253|BLACK+DECKER BDEJ...|           Tools|          5|    

In [11]:
# Retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_df = helpful_df.filter(helpful_df.vine == "N")
paid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21084288|R3QIQRG107AP19|B00OJ72LHK|     369815253|BLACK+DECKER BDEJ...|           Tools|          5|           38|         41|   Y|                N|Essential home ow...|We are not pros. ...| 2015-08-27|
|         US|   52512530|R1ZNBHGCYE8VN1|B00OJ72LHK|     369815253|BLACK+DECKER BDEJ...|           Tools|          5|    

In [14]:
#Determine the total number of reviews
print(filtered_df.count())


32975


In [16]:
 # Determine the number of 5-star reviews 

 five_stars = filtered_df.filter(filtered_df.star_rating == 5)
print(five_stars.count())

14141


In [38]:
# Determine the number of 'helpful' 5-star reviews
helpful_five_stars = helpful_df.filter(helpful_df.star_rating == 5)
print(five_stars.count())

13817


In [29]:
#the percentage of 5-star reviews for the two types of review (paid vs unpaid).
paid_five_stars = paid_df.filter(paid_df.star_rating == 5).count()
print(paid_five_stars)

149


In [26]:
print(unpaid_five_stars/13817)

0.010783817036983426


In [30]:
unpaid_five_stars = unpaid_df.filter(unpaid_df.star_rating == 5).count()
print(unpaid_five_stars)


13668


In [39]:
 # Determine the number of helpful paid reviews 
num_paid_reviews = helpful_five_stars.filter(helpful_five_stars.vine == "Y")

In [32]:
print(num_paid_reviews.count())

268


In [36]:
 # Determine the number of helpful unpaid reviews 
num_unpaid_reviews = helpful_df.filter(helpful_df.vine == "N")

In [37]:
print(num_unpaid_reviews.count())

29452


In [40]:
#Determine the number of helpful reviews
print(helpful_df.count())

29720
